In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
from obspy import read
from obspy import UTCDateTime
import pandas as pd
from scipy import stats
from scipy.fft import rfft, rfftfreq

### Read CC data, perform FFT, evenly pick 100 frequncies and amplitude, save as dataframe

In [ ]:
tree = "Forest/DP1/001_DAYS/11"
nontree = "Nonforest/DP1/001_DAYS/11"

def evenly_pick(lst, n):
    step = len(lst) // n
    return [lst[i] for i in range(0, len(lst), step)][:n]

data_row = []

for dir in [item for item in os.listdir(tree) if not item.startswith('.')]:
    cc_dir = tree + '/' + dir
    for file in os.listdir(cc_dir):
        cc_file = cc_dir + '/' + file
        data = read(cc_file)[0]
        amp = np.abs(rfft(data))
        freq = rfftfreq(len(data), 1/data.meta['sampling_rate'])
        mask = (freq > 10) & (freq < 100)
        filtered_freq = list(freq[mask])
        filtered_amp = list(amp[mask])
        selected_freq = evenly_pick(filtered_freq, 100)
        selected_amp = evenly_pick(filtered_amp, 100)
        data_row.append(selected_amp + ['Tree'])

for dir in [item for item in os.listdir(nontree) if not item.startswith('.')]:
    cc_dir = nontree + '/' + dir
    for file in os.listdir(cc_dir):
        cc_file = cc_dir + '/' + file
        data = read(cc_file)[0]
        amp = np.abs(rfft(data))
        freq = rfftfreq(len(data), 1/data.meta['sampling_rate'])
        mask = (freq > 10) & (freq < 100)
        filtered_freq = list(freq[mask])
        filtered_amp = list(amp[mask])
        selected_freq = evenly_pick(filtered_freq, 100)
        selected_amp = evenly_pick(filtered_amp, 100)
        data_row.append(selected_amp + ['Nontree'])

df = pd.DataFrame(data_row, columns = selected_freq +['results'])

def min_max_scale_row(row):
    min_val = row.min()
    max_val = row.max()
    return (row - min_val) / (max_val - min_val)

X = df.drop('results', axis = 1)
y = df.results
X = X.apply(min_max_scale_row, axis=1)

df = pd.concat([X, y], axis=1)

df.to_parquet('cc_data_fft_DP1.parquet')